In [0]:
%pip install great_expectations==1.10.0

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import shutil
import great_expectations as gx
import great_expectations.expectations as gxe

gx_root_dir = "/Volumes/traffic_dev_catalog/gx/gx_configs"

# 注意：dbutils 处理 Volume 路径时需要确保路径正确
# 如果直接传 /Volumes/... 不行，请尝试加上 /dbfs 前缀
path_to_delete = f"/dbfs{gx_root_dir}" if not gx_root_dir.startswith("/dbfs") else gx_root_dir

if os.path.exists(gx_root_dir):
    print(f"强制清空目录: {gx_root_dir}")
    # True 表示递归删除
    dbutils.fs.rm(gx_root_dir, True) 

# 重新创建
os.makedirs(gx_root_dir, exist_ok=True)

def get_context():
    return gx.get_context(context_root_dir=gx_root_dir)


In [0]:
context = get_context()

In [0]:
tables_to_validate = [
    {"table": "raw_roads", "suite": "raw_roads_suite"},
    {"table": "raw_traffic", "suite": "raw_traffic_suite"} 
]

for item in tables_to_validate:
    table_name = item["table"]
    suite_name = item["suite"]
    
    # 1.10+ 获取或创建 Suite 的标准写法
    try:
        # 使用 .suites.get 获取
        suite = context.suites.get(name=suite_name)
        print(f"✅ 已加载现有 Suite: {suite_name}")
    except Exception:
        # 使用 .suites.add 和 gx.ExpectationSuite 创建
        # 这会自动在 Files/gx_config/expectations/ 下生成对应的 .json 文件
        suite = context.suites.add(gx.ExpectationSuite(name=suite_name))
        print(f"✨ 已成功新建并持久化 Suite: {suite_name}")

print("\n所有 Suite 初始化完成。")

In [0]:
table_rules_mapping = {
    "raw_roads": [
        gxe.ExpectTableColumnsToMatchSet(
            column_set=[
                "Road_ID",
                "Road_Category_Id",
                "Road_Category",
                "Region_ID",
                "Region_Name",
                "Total_Link_Length_Km",
                "Total_Link_Length_Miles",
                "All_Motor_Vehicles",
                "_dq_batch_id"
            ],
            exact_match=True 
        ),
        gxe.ExpectColumnValuesToNotBeNull(column="Road_ID"),
        gxe.ExpectColumnValuesToBeBetween(column="Total_Link_Length_Km", min_value=0),
        gxe.ExpectColumnValuesToBeBetween(column="Road_Category_Id", min_value=1, max_value=5)
    ],
    "raw_traffic": [
        gxe.ExpectTableColumnsToMatchSet(
            column_set=[
                "Record_ID",
                "Count_point_id",
                "Direction_of_travel",
                "Year",
                "Count_date",
                "hour",
                "Region_id",
                "Region_name",
                "Local_authority_name",
                "Road_name",
                "Road_Category_ID",
                "Start_junction_road_name",
                "End_junction_road_name",
                "Latitude",
                "Longitude",
                "Link_length_km",
                "Pedal_cycles",
                "Two_wheeled_motor_vehicles",
                "Cars_and_taxis",
                "Buses_and_coaches",
                "LGV_Type",
                "HGV_Type",
                "EV_Car",
                "EV_Bike",
                "Extract_Time",
                "_dq_batch_id"
            ],
            exact_match=True 
        ),
        gxe.ExpectColumnValuesToNotBeNull(column="Record_ID"),
        gxe.ExpectColumnValuesToMatchStrftimeFormat(
            column="Count_date",
            strftime_format="%m/%d/%Y %H:%M"  
        ),
        gxe.ExpectColumnValuesToBeBetween(column="hour", min_value=1, max_value=24),
        gxe.ExpectColumnValuesToBeBetween(column="Road_Category_Id", min_value=1, max_value=5)
    ]
}

In [0]:
def initialize_all_suites(context, rules_mapping):
    for table_name, expectations in rules_mapping.items():
        suite_name = f"{table_name}_suite"
        
        # 1. 获取或创建 Suite
        try:
            suite = context.suites.get(name=suite_name)
            suite.expectations = [] 
            print(f"🔄 更新现有 Suite: {suite_name}")
        except Exception:
            suite = context.suites.add(gx.ExpectationSuite(name=suite_name))
            print(f"✨ 创建新 Suite: {suite_name}")

        # 2. 为该表添加定义的每一条规则
        for exp in expectations:
            # add_expectation 在 1.x 中会自动去重（如果规则完全一样）
            suite.add_expectation(exp)
        
        print(f"   已添加 {len(expectations)} 条规则到 {suite_name}")

In [0]:
initialize_all_suites(context, table_rules_mapping)

In [0]:
%sql
-- 1. 先彻底删除旧表，确保没有任何旧的 Schema 残留
DROP TABLE IF EXISTS traffic_dev_catalog.gx.data_quality_quarantine;

-- 2. 重新创建符合新代码逻辑的表
CREATE TABLE traffic_dev_catalog.gx.data_quality_quarantine (
    table_name STRING COMMENT 'The name of the source table where the data originated',
    -- 修改点：改名为 gx_batch_id，并改为 STRING 类型以解决合并冲突
    gx_batch_id STRING COMMENT 'The identifier for the GX validation run (casted to string)',
    violated_rules STRING COMMENT 'A list or description of the rules that failed validation',
    raw_data STRING COMMENT 'The original record stored in JSON format',
    ingestion_time TIMESTAMP COMMENT 'The timestamp when the record was quarantined'
)
USING DELTA
TBLPROPERTIES (
    'delta.columnMapping.mode' = 'name',
    'delta.minReaderVersion' = '2',
    'delta.minWriterVersion' = '5'
)
COMMENT 'Universal data quality quarantine table for storing records that failed GX validation';